In [17]:
import IPython, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa, cmath,math
from IPython.display import Audio
 

In [18]:
def estimated_beat(wav_file, hop_size, start_time, end_time, sr, start_bpm):
    signal, sr = librosa.load(wav_file, sr)
    hop_length = 512
    start = start_time
    end = end_time
    tempo, beats = librosa.beat.beat_track(y=signal, sr=sr, onset_envelope=None, hop_length=hop_size,
                start_bpm=start_bpm, tightness=100, trim=True, bpm=None)
    onset_env = librosa.onset.onset_strength(y=signal, sr=sr, S=None, lag=1, max_size=1, detrend=False, 
                                 center=True, feature=None, aggregate=None)
    plt.figure()
    plt.plot(librosa.util.normalize(onset_env), label='Onset strength')
    plt.vlines(beats, 0, 1, alpha=0.5, color='r',
                linestyle='--', label='Beats')
    plt.legend(frameon=True, framealpha=0.75)
    # Limit the plot to a 15-second window
    plt.xlim([start * sr / hop_length, end * sr / hop_length])
    plt.xticks(np.linspace(start, end, 5) * sr / hop_length,
                np.linspace(start, end, 5))
    plt.xlabel('Time (s)')
    plt.tight_layout()
    return beats

def calculate_rank(original_beats, test_beats, sigma):
    score_array = np.zeros(len(original_beats))
    for i in range (0, len(original_beats)):
        for k in range (0, len(test_beats)):
            if ((original_beats[i] + sigma > test_beats[k]) and (original_beats[i] - sigma < test_beats[k])):
                score_array[i] = 1.0
    score = np.sum(score_array)/float(len(score_array))
    return score



In [19]:
hop_size = 512
start_time = 30
end_time = 45
sr = 44100
start_bpm = 120
sigma = 10 
# Sigma is how much off the beat the person is allowed to be
test_beat_frames = estimated_beat('test.wav', hop_size, start_time, end_time, sr, start_bpm)
original_beat_frames = estimated_beat('fireflies.wav', hop_size, start_time, end_time, sr, start_bpm)
plt.show()
test_beats = librosa.frames_to_samples(test_beat_frames, hop_length=hop_size)
original_beats = librosa.frames_to_samples(original_beat_frames, hop_length=hop_size)
score = calculate_rank(original_beats, test_beats, sigma)